In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import transformers as ppb 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification


**Loading Dataset and Pre-processing**

The dataset that we are using is the amazon mobile/electronics product reviews which is a subset of the large Amazon Product review. The dataset is already stored in the TensorFlow database and can be loaded directly using the ‘tfds‘ API from Tensorflow. Once loaded you’ll have to convert it into a pandas data frame using ‘tfds.as_dataframe‘ API.

In [ ]:
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

<_OptionsDataset element_spec={'data': {'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'helpful_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'marketplace': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_category': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_parent': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_body': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_date': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_headline': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'star_rating': TensorSpec(shape=(), dtype=tf.int32, name=None), 'total_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'verified_purchase': TensorSpec(shape=(), dtype=tf.int64, name=None), 'vine': TensorSpec(shape=(), dtype=

In [ ]:
#convert the dataset into a pandas dataframe
df = tfds.as_dataframe(ds)

The dataset consists of several columns ranging from Product ID to reviews, heading, and star rating provided by the customer. As we are only interested in the reviews and the corresponding rating provided by the customer, we are going to drop the other feature columns.

The rating provided by the customer is on a scale of 1-5( 5 being the highest). As we are going to implement a binary classification model, we will need to convert these ratings into 2 categories,i.e 1 and 0. Ratings above and equal to 3 will be labeled as Positive(1) and below 3 will be negative(0). The following code will help us implement these steps.

In [ ]:
df["Sentiment"] = df["data/star_rating"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df['short_review'] =df['data/review_body'].str.decode("utf-8")
df = df[["short_review", "Sentiment"]]

In our dataset, we still have a large corpus of reviews provided by the customer. Since BERT requires high computational power and takes a large amount of time to train on the data frame, we will drop some rows from our dataset to reduce the time for training on it.

In [ ]:
# Dropping last n rows using drop
n = 34975
df.drop(df.tail(n).index,
        inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df.shape

(70000, 2)

We will convert our feature column and label into a set of lists as that’s how our Tokenizer wants our data. 

In [ ]:
reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()

#Assign tokenizer object to the tokenizer class





In [ ]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
MAX_LEN = 20
inputs = tokenizer(reviews, max_length=MAX_LEN, truncation=True, padding=True)

In [ ]:
def construct_encodings(x, tkzr, max_len, trucation=True, padding=True):
    return tkzr(x, max_length=max_len, truncation=trucation, padding=padding)
    
encodings = construct_encodings(reviews, tokenizer, max_len=MAX_LEN)

The Tensorflow API provides a seemingly easy way to build data pipelines. Using “from-Tensor-Slices“, we can easily combine our features tokens and labels into a dataset.

In [ ]:
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))
    
tfdataset = construct_tfdataset(encodings, labels)

In [ ]:
TEST_SPLIT = 0.2
BATCH_SIZE = 16

train_size = int(len(reviews) * (1-TEST_SPLIT))

tfdataset = tfdataset.shuffle(len(reviews))
tfdataset_train = tfdataset.take(train_size)
tfdataset_test = tfdataset.skip(train_size)

tfdataset_train = tfdataset_train.batch(BATCH_SIZE)
tfdataset_test = tfdataset_test.batch(BATCH_SIZE)

In [ ]:
N_EPOCHS = 3
MODEL_NAME = 'distilbert-base-uncased'
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_59', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.fit(tfdataset_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS)

Epoch 1/3
3500/3500 [==============================] - 214s 59ms/step - loss: 0.3202 - accuracy: 0.8602
Epoch 2/3
3500/3500 [==============================] - 205s 59ms/step - loss: 0.2546 - accuracy: 0.8921
Epoch 3/3
3500/3500 [==============================] - 205s 59ms/step - loss: 0.2021 - accuracy: 0.9186


In [ ]:
evaluation = model.evaluate(tfdataset_test, return_dict=True, batch_size=BATCH_SIZE)

875/875 [==============================] - 18s 19ms/step - loss: 0.1259 - accuracy: 0.9534


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Dataset/sentiment")

In [ ]:
test_sentence = "This product is really amaizing"
predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

In [ ]:
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

Positive


**Model Training and optimization**

In our application, we are going to use TFDistilBertForSequenceClassification for the sentiment analysis and put the ‘num-labels’ parameter equal to 2 as we are doing a binary classification.

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

We don’t need to put any additional layers and using a Hugging face transformer, we can now train our model with the following configuration:

1. epochs: 2
2. Batch size: 16
3. Learning rate (Adam): 5e-5 (0.00005)